# Report

# Environment Setup and Import

In [ ]:
!pip install --quiet -r ../../requirements.txt

In [8]:
# Standard Libraries
import json
import shutil
import urllib.request as urlrequest
from collections import Counter
from pathlib import Path
from pprint import pprint
from zipfile import ZipFile
# 3rd-party Libraries
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import make_scorer,recall_score, roc_curve, roc_auc_score
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from tensorflow import keras
import os
print(os.getcwd())

2023-07-06 06:28:45.665786: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-06 06:28:49.573831: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-06 06:28:49.575264: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 06:28:55.341342: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


/workspaces/opengamedata-samples/AQUALAB/templates


In [4]:
from OGDUtils.general.import_utils import FileAPI, FileTypes

Change pandas `max_rows` and `max_columns`

In [ ]:
pd.options.display.max_columns = 4000 #@param {type:"integer"}
pd.options.display.max_rows = 60 #@param {type:"integer"}

## Load Data

#### Please change the settings below to configure which game/server location you want to use.

<!-- `ogd_server` : The file server that contains the actual data files -->
* `api_server` : The server hosting the API instance.
* `api_path` : The "path" portion of the API URL. Change this if you want to access a different version of the API.
* `game_id` : The game whose data you want to access.
* `mode` : The kind of data you want to retrieve. Should be one of `'players'`, `'population'`, `'sessions'`, or `'events'`

Server Settings

In [10]:
game_id = 'AQUALAB'
mode    = FileTypes.PLAYER

In [9]:
months_list = FileAPI.GetAvailableMonths(game_id)
print(f"The available months are:")
pprint(months_list, compact=True)

The available months are:
['4/2021', '5/2021', '6/2021', '7/2021', '8/2021', '9/2021', '10/2021',
 '11/2021', '12/2021', '1/2022', '2/2022', '3/2022', '4/2022', '5/2022',
 '6/2022', '7/2022', '8/2022', '9/2022', '10/2022', '11/2022', '12/2022',
 '1/2023', '2/2023', '3/2023', '4/2023', '5/2023', '6/2023']


Dataset Selection & Download

In [16]:
month = 2
year = 2023

# download the file if it doesn't exist.
zip_file, dataset_name = FileAPI.DownloadZippedDataset(game_id, month, year, mode)

Didn't find the file AQUALAB_20230201_to_20230228_5de1a1f_player-features.zip locally, downloading from http://ogd-services.fielddaylab.wisc.edu/...
Successfully downloaded a copy of the file.


For zip file, depending what you are working with

In [17]:
zip_name = zip_file.filename
print(zip_name)

AQUALAB_20230201_to_20230228_5de1a1f_player-features.zip


In [18]:
raw_data = pd.DataFrame()

# Open TSV from the zip file.
tsv_name = f"{zip_name[:zip_name.rfind('.')]}.tsv"
with zip_file.open(f"{dataset_name}/{tsv_name}") as tsv_file:
    raw_data = pd.read_csv(tsv_file, sep='\t')
    data_readme = zip_file.read(f"{dataset_name}/readme.md")
zip_file.close()

In [19]:
typed_data = raw_data.copy()
for col, dtype in zip(raw_data, raw_data.dtypes):
  if dtype == "string":
    try:
      typed_data[col] = pd.to_timedelta(raw_data[col])
    except:
      try:
        typed_data[col] = pd.to_numeric(raw_data[col])
      except:
        pass
  elif dtype == "object":
      try:
        typed_data[col] = pd.to_numeric(raw_data[col])
      except:
        pass

print(f"This dataframe contains data of the following types:\n{typed_data.dtypes.unique()}\n")
print(f"Per feature, the data types are:\n{typed_data.dtypes}")

This dataframe contains data of the following types:
[dtype('O') dtype('int64') dtype('float64')]

Per feature, the data types are:
PlayerID                                     object
SessionCount                                  int64
ActiveTime                                   object
JobsCompleted                                object
SessionDiveSitesCount                         int64
                                             ...   
job60_JobsAttempted-num-completes             int64
job60_JobsAttempted-percent-complete          int64
job60_JobsAttempted-avg-time-per-attempt    float64
job60_JobsAttempted-std-dev-per-attempt       int64
job60_JobsAttempted-job-difficulties         object
Length: 1359, dtype: object


Sample Rows

In [20]:
rows_to_show = 10
typed_data.head(rows_to_show)

,PlayerID,SessionCount,ActiveTime,JobsCompleted,SessionDiveSitesCount,SessionGuideCount,SessionHelpCount,SessionID,SessionJobsCompleted,SwitchJobsCount,TopJobCompletionDestinations,TopJobSwitchDestinations,TotalArgumentationTime,TotalDiveTime,TotalExperimentationTime,job0_JobActiveTime,job1_JobActiveTime,job2_JobActiveTime,job3_JobActiveTime,job4_JobActiveTime,job5_JobActiveTime,job6_JobActiveTime,job7_JobActiveTime,job8_JobActiveTime,job9_JobActiveTime,job10_JobActiveTime,job11_JobActiveTime,job12_JobActiveTime,job13_JobActiveTime,job14_JobActiveTime,job15_JobActiveTime,job16_JobActiveTime,job17_JobActiveTime,job18_JobActiveTime,job19_JobActiveTime,job20_JobActiveTime,job21_JobActiveTime,job22_JobActiveTime,job23_JobActiveTime,job24_JobActiveTime,...,job56_JobsAttempted,job56_JobsAttempted-job-name,job56_JobsAttempted-num-starts,job56_JobsAttempted-num-completes,job56_JobsAttempted-percent-complete,job56_JobsAttempted-avg-time-per-attempt,job56_JobsAttempted-std-dev-per-attempt,job56_JobsAttempted-job-difficulties,job57_JobsAttempted,job57_JobsAttempted-job-name,job57_JobsAttempted-num-starts,job57_JobsAttempted-num-completes,job57_JobsAttempted-percent-complete,job57_JobsAttempted-avg-time-per-attempt,job57_JobsAttempted-std-dev-per-attempt,job57_JobsAttempted-job-difficulties,job58_JobsAttempted,job58_JobsAttempted-job-name,job58_JobsAttempted-num-starts,job58_JobsAttempted-num-completes,job58_JobsAttempted-percent-complete,job58_JobsAttempted-avg-time-per-attempt,job58_JobsAttempted-std-dev-per-attempt,job58_JobsAttempted-job-difficulties,job59_JobsAttempted,job59_JobsAttempted-job-name,job59_JobsAttempted-num-starts,job59_JobsAttempted-num-completes,job59_JobsAttempted-percent-complete,job59_JobsAttempted-avg-time-per-attempt,job59_JobsAttempted-std-dev-per-attempt,job59_JobsAttempted-job-difficulties,job60_JobsAttempted,job60_JobsAttempted-job-name,job60_JobsAttempted-num-starts,job60_JobsAttempted-num-completes,job60_JobsAttempted-percent-complete,job60_JobsAttempted-avg-time-per-attempt,job60_JobsAttempted-std-dev-per-attempt,job60_JobsAttempted-job-difficulties
0,NaN,1,No events,[],0,0,0,player,0,0,{},{},0:00:00,0:00:00,0:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,56,arctic-endangered-seals,0,0,0.0,0.0,0,"{""experimentation"": 5, ""modeling"": 4, ""argumen...",57,bayou-blue-waters,0,0,0,0.0,0,"{""experimentation"": 5, ""modeling"": 0, ""argumen...",58,bayou-dirty-detritus,0,0,0,0.0,0,"{""experimentation"": 3, ""modeling"": 0, ""argumen...",59,bayou-shrimp-yields,0,0,0,0.0,0,"{""experimentation"": 0, ""modeling"": 0, ""argumen...",60,final-final,0,0,0,0.0,0,"{""experimentation"": 0, ""modeling"": 2, ""argumen..."
1,AbsorbedComma,2,23:22:16,[],9,0,5,player,0,2,{},{},0:00:00,0:02:26,0:00:00,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4805.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,56,arctic-endangered-seals,0,0,0.0,0.0,0,"{""experimentation"": 5, ""modeling"": 4, ""argumen...",57,bayou-blue-waters,0,0,0,0.0,0,"{""experimentation"": 5, ""modeling"": 0, ""argumen...",58,bayou-dirty-detritus,0,0,0,0.0,0,"{""experimentation"": 3, ""modeling"": 0, ""argumen...",59,bayou-shrimp-yields,0,0,0,0.0,0,"{""experimentation"": 0, ""modeling"": 0, ""argumen...",60,final-final,0,0,0,0.0,0,"{""experimentation"": 0, ""modeling"": 2, ""argumen..."
2,AbsorbedForest,1,0:07:40,[],0,0,0,player,0,1,{},{},0:00:00,0:00:00,0:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,56,arctic-endangered-seals,0,0,0.0,0.0,0,"{""experimentation"": 5, ""modeling"": 4, ""argumen...",57,bayou-blue-waters,0,0,0,0.0,0,"{""experimentation"": 5, ""modeling"": 0, ""argumen...",58,bayou-dirty-detritus,0,0,0,0.0,0,"{""experimentation"": 3, ""modeling"": 0, ""argumen...",59,bayou-shrimp-yields,0,0,0,0.0,0,"{""experimentation"": 0, ""modeling"": 0, ""argumen...",60,final-final,0,0,0,0.0,0,"{""experimentation"": 0, ""modeling"": 2,

Summary

In [ ]:
typed_data.describe(include='all')

In [ ]:
#@markdown Do you want to filter player list?
Answer  = False#@param {type:"boolean"}
#@markdown Please change the file path if your player list is located differently.
PL_Path = "PlayerList.txt"#@param {type:"string"}
if Answer:
  player_list = pd.read_csv(JUPYTER_DIR / Path(PL_Path), sep='t', names=["Player"])
  raw_data = raw_data[raw_data.PlayerId.isin()]

Workable Dataframe After Dropping NAs

In [ ]:
df_session = typed_data.dropna()


#### Type Conversions

In [ ]:
for col, dtype in zip(df_session, df_session.dtypes):
  if dtype == "string":
    try:
      df_session[col] = pd.to_timedelta(df_session[col])
    except:
      pass

In [ ]:
#Parameters for cleaning

#@markdown Type a perjob feature name here
Interested_perjobfeature = 'ArgumentationTime'#@param {type:"string"}
#@markdown Does it have a corresponding aggreation feature with same name? Check it if yes.
Answer  = True#@param {type:"boolean"}
id_columns = pd.Index(["SessionID", "PlayerID"])

In [ ]:
# change interested columns into strings
# #dtypes for all columns
df_Interested = df_session.convert_dtypes()

In [ ]:
#if dtype is string, change to timedelta if possible
for col, dtype in zip(df_Interested, df_Interested.dtypes):
  if dtype == "string":
    try:
      df_Interested[col] = pd.to_timedelta(df_Interested[col])
    except:
      pass

In [ ]:
#data frame we want to change to time delta
target_columns = df_Interested.columns[df_Interested.columns.str.contains(Interested_perjobfeature)]
df_Interested = df_Interested[id_columns.append(target_columns)].copy()
df_Interested.head()

In [ ]:
#turn time into seconds
for col in target_columns:
  df_Interested[col] = df_Interested[col].dt.total_seconds()
df_Interested.dtypes

In [ ]:
#find outliers of what columns
outlier = 4000 #@param {type:"integer"}
bigoutliers = np.where(df_Interested[target_columns].ge(outlier)) 
#df of outliers
df_Interested[target_columns].iloc[bigoutliers]

In [ ]:
df_Interested.drop(df_Interested.iloc[:, :2], inplace = True, axis = 1)

# Overview

## Data Overview

In [ ]:
#split depending on prefix, clean columns
cols_split = []
for col in df_session.columns:
  pieces = col.split('_')
  prefix = pieces[0]
  if prefix.startswith('job') or prefix.startswith('int') or prefix.startswith('obj') \
  or prefix.startswith('SA')  or prefix.startswith('Q'):
    cols_split.append(pieces[1])
  else:
    cols_split.append(col)
col_set = Counter(cols_split)
print(f"{len(df_session.columns)} Columns in {len(col_set)} groups:")
pprint(col_set, compact=False)

In [ ]:
#data types in each column
df_session.dtypes

In [ ]:
df_session.head(rows_to_show)

In [ ]:
#Data frame to analyze
target = 'job' #@param {type:"string"}
target_columns = df_session.columns[~df_session.columns.str.contains(target)]
df_Aggregate = df_session[target_columns].copy()
df_Aggregate

# Visualizations

Amount of players

In [ ]:
#Amount of unique PlayerIDs - amount of players
len(df_Aggregate['PlayerID'])

How long did they play?

In [ ]:
df_column = df_session['UserTotalSessionDuration'] #column must be integers

In [ ]:
#jobs completed
f, ax = plt.subplots()
ax.hist(df_column, bins=30)
ax.axvline(x=df_column.mean(), linestyle='--', color='k', label="Median")
ax.text(x=df_column.mean() + 0.25, y=500, s="Mean", rotation=-90)
ax.set_xlabel("Seconds spent playing")
ax.set_title("Histogram for Time Spent Playing")
print(f'The average time spent was {df_column.mean()} seconds')

How far did the players get?

In [ ]:

#@parameter
df_column = df_session['SessionJobsCompleted'] #column must be integers

In [ ]:
#jobs completed
f, ax = plt.subplots()
ax.hist(df_column, bins=30)
ax.axvline(x=df_column.mean(), linestyle='--', color='k', label="Median")
ax.text(x=df_column.mean() + 0.25, y=500, s="Mean", rotation=-90)
ax.set_xlabel("Number of sessions")
ax.set_title("Histogram for Jobs Completed")
print(f'The average amount of levels completed: {df_column.mean()}')

X vs Y Scatter Plot

In [ ]:
#scatter plot columns
col1 = 'SessionHelpCount' #@param{type:"string"}
col2 = 'SessionJobsCompleted'#@param{type:"string"}

In [ ]:
plt.scatter(df_session[col1], df_session[col2])
m, b = np.polyfit(df_session[col1], df_session[col2], 1)
plt.plot(df_session[col1], m*df_session[col1]+b, color = 'red')
plt.xlabel(col1)
plt.ylabel(col2)
print(f'Slope of line: {(m*df_session[col1]+b).mean()}')

Amount of Sessions vs Avg Length of Session

In [ ]:
#scatter plot columns
col1 = 'SessionCount' #@param{type:"string"}
col2 = 'UserAvgSessionDuration'#@param{type:"string"}

In [ ]:
plt.scatter(df_Aggregate[col1], df_Aggregate[col2])
m, b = np.polyfit(df_Aggregate[col1], df_Aggregate[col2], 1)
plt.plot(df_Aggregate[col1], m*df_Aggregate[col1]+b, color = 'red')
plt.xlabel(col1)
plt.ylabel(col2)
print(f'Slope of line: {(m*df_session[col1]+b).mean()}')

In [ ]:
target_cols = ['SessionDiveSitesCount', 'SessionGuideCount', 'SessionHelpCount', 'SessionJobsCompleted', 'SwitchJobsCount', 'UserAvgSessionDuration', 'UserTotalSessionDuration']
        
fig, ax = plt.subplots(figsize=(20,20))         # Sample figsize in inches
df_Aggregate[target_cols].hist(ax=ax)

Heatmap across columns x,y,z...

In [ ]:
features1= ['SessionHelpCount','SessionJobsCompleted','UserTotalSessionDuration', 'SwitchJobsCount']

sns.heatmap(data = df_Aggregate[features1], cmap = 'tab20')